# Supervised classification of anatomy on orthopedic X-ray data

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import data, exposure
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import numpy as np
import os
from matplotlib import pyplot as plt
from skimage import exposure
from skimage.filters import rank
from skimage.morphology import disk

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')

## Class Imbalance for only

In [ ]:
base_dir = '/kaggle/input/mura12/MURA-v1.1/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')

labels = ['XR_ELBOW','XR_FINGER','XR_FOREARM','XR_HAND','XR_HUMERUS','XR_SHOULDER','XR_WRIST']

print("Train set:\n========================================")
num_XR_ELBOW = len(os.listdir(os.path.join(train_dir, 'XR_ELBOW')))
num_XR_FINGER = len(os.listdir(os.path.join(train_dir, 'XR_FINGER')))
num_XR_FOREARM = len(os.listdir(os.path.join(train_dir, 'XR_FOREARM')))
num_XR_HAND = len(os.listdir(os.path.join(train_dir, 'XR_HAND')))
num_XR_HUMERUS = len(os.listdir(os.path.join(train_dir, 'XR_HUMERUS')))
num_XR_SHOULDER = len(os.listdir(os.path.join(train_dir, 'XR_SHOULDER')))
num_XR_WRIST = len(os.listdir(os.path.join(train_dir, 'XR_WRIST')))
print(f"XR_ELBOW={num_XR_ELBOW}")
print(f"XR_XR_FINGER={num_XR_FINGER}")
print(f"XR_FOREARM={num_XR_FOREARM}")
print(f"XR_XR_HAND={num_XR_HAND}")
print(f"XR_HUMERUS={num_XR_HUMERUS}")
print(f"XR_XR_SHOULDER={num_XR_SHOULDER}")
print(f"XR_XR_WRIST={num_XR_WRIST}")

print("Valid set:\n========================================")
print(f"XR_ELBOW={len(os.listdir(os.path.join(validation_dir, 'XR_ELBOW')))}")
print(f"XR_XR_FINGER={len(os.listdir(os.path.join(validation_dir, 'XR_FINGER')))}")
print(f"XR_FOREARM={len(os.listdir(os.path.join(validation_dir, 'XR_FOREARM')))}")
print(f"XR_XR_HAND={len(os.listdir(os.path.join(validation_dir, 'XR_HAND')))}")
print(f"XR_HUMERUS={len(os.listdir(os.path.join(validation_dir, 'XR_HUMERUS')))}")
print(f"XR_XR_SHOULDER={len(os.listdir(os.path.join(validation_dir, 'XR_SHOULDER')))}")
print(f"XR_XR_WRIST={len(os.listdir(os.path.join(validation_dir, 'XR_WRIST')))}")


weight_for_num_XR_ELBOW = num_XR_ELBOW / (num_XR_ELBOW + num_XR_FINGER + num_XR_FOREARM + num_XR_HAND + num_XR_HUMERUS + num_XR_SHOULDER + num_XR_ELBOW)
weight_for_num_XR_FINGER = num_XR_FINGER / (num_XR_ELBOW + num_XR_FINGER + num_XR_FOREARM + num_XR_HAND + num_XR_HUMERUS + num_XR_SHOULDER + num_XR_ELBOW)
weight_for_num_XR_FOREARM= num_XR_FOREARM / (num_XR_ELBOW + num_XR_FINGER + num_XR_FOREARM + num_XR_HAND + num_XR_HUMERUS + num_XR_SHOULDER + num_XR_ELBOW)
weight_for_num_XR_HAND = num_XR_HAND / (num_XR_ELBOW + num_XR_FINGER + num_XR_FOREARM + num_XR_HAND + num_XR_HUMERUS + num_XR_SHOULDER + num_XR_ELBOW)
weight_for_num_XR_HUMERUS = num_XR_HUMERUS / (num_XR_ELBOW + num_XR_FINGER + num_XR_FOREARM + num_XR_HAND + num_XR_HUMERUS + num_XR_SHOULDER + num_XR_ELBOW)
weight_for_num_XR_SHOULDER = num_XR_SHOULDER / (num_XR_ELBOW + num_XR_FINGER + num_XR_FOREARM + num_XR_HAND + num_XR_HUMERUS + num_XR_SHOULDER + num_XR_ELBOW)
weight_for_num_XR_WRIST = num_XR_WRIST / (num_XR_ELBOW + num_XR_FINGER + num_XR_FOREARM + num_XR_HAND + num_XR_HUMERUS + num_XR_SHOULDER + num_XR_ELBOW)



class_weight = {0: weight_for_num_XR_ELBOW,
                1: weight_for_num_XR_FINGER,
                2: weight_for_num_XR_FOREARM,
                3: weight_for_num_XR_HAND,
                4:weight_for_num_XR_HUMERUS,
                5: weight_for_num_XR_SHOULDER,
                6: weight_for_num_XR_WRIST,
                }

print(f"Weight for class weight_for_num_XR_ELBOW: {weight_for_num_XR_ELBOW:.2f}")
print(f"Weight for class weight_for_num_XR_FINGER: {weight_for_num_XR_FINGER:.2f}")
print(f"Weight for class weight_for_num_XR_FOREARM: {weight_for_num_XR_FOREARM:.2f}")
print(f"Weight for class weight_for_num_XR_HAND: {weight_for_num_XR_HAND:.2f}")
print(f"Weight for class weight_for_num_XR_HUMERUS: {weight_for_num_XR_HUMERUS:.2f}")
print(f"Weight for class weight_for_num_XR_SHOULDER: {weight_for_num_XR_SHOULDER:.2f}")
print(f"Weight for class weight_for_num_XR_WRIST: {weight_for_num_XR_WRIST:.2f}")

### Implemented Densenet

In [ ]:
with strategy.scope():
    def H(inputs, num_filters, dropout_rate):
        x = tf.keras.layers.BatchNormalization(epsilon=eps)(inputs)
        x = tf.keras.layers.Activation('relu')(x)
        x = tf.keras.layers.ZeroPadding2D((1, 1))(x)
        x = tf.keras.layers.Conv2D(num_filters, kernel_size=(3, 3), use_bias=False, kernel_initializer='he_normal')(x)
        x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
        return x


    def transition(inputs, num_filters, compression_factor, dropout_rate):
        x = tf.keras.layers.BatchNormalization(epsilon=eps)(inputs)
        x = tf.keras.layers.Activation('relu')(x)
        num_feature_maps = inputs.shape[1]  # The value of 'm'

        x = tf.keras.layers.Conv2D(np.floor(compression_factor * num_feature_maps).astype(np.int),
                                   kernel_size=(1, 1), use_bias=False, padding='same', kernel_initializer='he_normal',
                                   kernel_regularizer=tf.keras.regularizers.l2( 1e-4 ))(x)
        x = tf.keras.layers.Dropout(rate=dropout_rate)(x)

        x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(x)
        return x


    def dense_block(inputs, num_layers, num_filters, growth_rate, dropout_rate):
        for i in range(num_layers):  # num_layers is the value of 'l'
            conv_outputs = H(inputs, num_filters, dropout_rate)
            inputs = tf.keras.layers.Concatenate()([conv_outputs, inputs])
            num_filters += growth_rate  # To increase the number of filters for each layer.
        return inputs, num_filters


    input_shape = (224, 224, 3)
    num_blocks = 3
    num_layers_per_block = 4
    growth_rate = 16
    dropout_rate = 0
    compress_factor = 0.5
    eps = 1.1e-5

    num_filters = 16

    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(num_filters, kernel_size=(3, 3), use_bias=False, kernel_initializer='he_normal',
                               kernel_regularizer=tf.keras.regularizers.l2( 1e-4 ))(inputs)

    for i in range(num_blocks):
        x, num_filters = dense_block(x, num_layers_per_block, num_filters, growth_rate, dropout_rate)
        x = transition(x, num_filters, compress_factor, dropout_rate)

    x = tf.keras.layers.Flatten()(x)

    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(7)(x)
    outputs = tf.keras.layers.Activation('softmax')(x)
    model = tf.keras.models.Model(inputs, outputs)
    model.summary()
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=3, mode='min')
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(lr=0.0007),
                  metrics=['categorical_accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])  
    dot_img_file = 'model_1.png'
    tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)
    

### Normal Convolution network

In [ ]:
with strategy.scope():
    model = tf.keras.models.Sequential([
        # Note the input shape is the desired size of the image 150x150 with 3 bytes color
        # This is the first convolution
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The third convolution
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The fourth convolution
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(7, activation='softmax')])
    model.summary()
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(lr=0.0007),
                  metrics=['categorical_accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])  

### Pre train densenet

In [ ]:
with strategy.scope():
    vgg16 = tf.keras.applications.VGG16(
    include_top=False, weights='imagenet',
    input_shape=(224, 224, 3), pooling=None, classes=7,
    classifier_activation='softmax'
    )
    
    for layer in vgg16.layers:
        layer.trainable = False
    x = tf.keras.layers.Flatten()(vgg16.output)
    x = tf.keras.layers.Dense(7, activation = 'softmax')(x) 
    model = tf.keras.models.Model(inputs = vgg16.input, outputs = x)
    model.summary()
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(lr=0.0007),
                  metrics=['categorical_accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])  

#  Saving the best weights
When using 'Callback' and 'ModelCheckpoint' utilities of Keras, we can save the model with the best weight

In [ ]:
#Save the model during training 
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



scoreSeg = model.evaluate_generator(validation_generator, 400)

save_at = "/kaggle/working/model_Mura.hdf5"
save_best = ModelCheckpoint (save_at, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='max')



### Pre train densenet

In [ ]:
  with strategy.scope():
    model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    x = model_d.output
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dropout(0.5)(x)

    preds = Dense(8, activation='softmax')(x)  # FC-layer

## Training Generater and complite model 

In [ ]:
from skimage import data, exposure
def AHE(img):
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    
    return img_adapteq
def image(img):
    return tf.image.per_image_standardization(img)
    
def equalization(img):
    # some images are just one color, so they gerenate a divide by zero error
    #     so return original image if the min and max values are the same
    # print("image shape",img.shape)
    if (np.max(img) == np.min(img) ):
        return img
    # Equalization
    img_equalized = exposure.equalize_hist(img)
    return img_equalized

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    samplewise_center = True,
    featurewise_std_normalization= False,
    samplewise_std_normalization=True
)
test_datagen = ImageDataGenerator(rescale=1. / 255,
                  featurewise_std_normalization=False,
                  samplewise_std_normalization=True,
                  samplewise_center =True                                  )
batch_size = 8
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,  # This is the source directory for training images
    target_size=(224, 224),  # All images will be resized to 150x150
    batch_size=batch_size,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='categorical',shuffle=False)
print("train_generator", train_generator.total_batches_seen)
# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',shuffle=False)

In [ ]:
imgs, labels = next(validation_generator)
Y_pred = np.round(model.predict(imgs))

np.random.seed(87)
for rand_num in np.random.randint(0, len(Y_pred), 5):
    plt.figure()
    plt.imshow(X_test[rand_num].reshape(100, 100)), plt.axis('off')
    if np.where(Y_pred[rand_num] == 1)[0].sum() == np.where(Y_test[rand_num] == 1)[0].sum():
        plt.title(encoder.classes_[np.where(Y_pred[rand_num] == 1)[0].sum()], color='g')
    else :
        plt.title(encoder.classes_[np.where(Y_pred[rand_num] == 1)[0].sum()], color='r')

# Verify Images
In this section, we verify the images are loading correctly, are readable and deliver reasonably understandable results. Batch size is 8 so 8 images are displayed

In [ ]:
a, b = next(validation_generator)
tg_class = {v:k for k,v in validation_generator.class_indices.items()}
fig, m_axs = plt.subplots(3, 3, figsize = (12, 12))
for c_ax, c_img, c_lab in zip(m_axs.flatten(), a, b):
    c_ax.imshow(c_img[:,:,1], cmap = 'bone')
    c_ax.axis('off')
    c_ax.set_title(tg_class[np.argmax(c_lab)])


## Setting seeds for reproducibility

In [ ]:
# seed = 232
# np.random.seed(seed)
# tf.random.set_seed(seed)

### Agumented Image

In [ ]:
def plotImages(images_arr):
    plt.figure()
    fig, axes = plt.subplots(1,6, figsize=(40,40))

    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

import numpy as np
def plots(ims, figsize=(40,40), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1

    for i in range(len(ims)):
        sp = f.add_subplot(cols, rows, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=12)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
#Check the training set (with batch of 10 as defined above
imgs, labels = next(train_generator)
print("train_generator.classes",train_generator.classes)
print("train_generator.indice",train_generator.class_indices)
#Images are shown in the output
plots(imgs, titles=labels)

augmented_images = [train_generator[1][1][1] for i in range(6)]
plotImages(imgs)
print("This is the label \n",labels)

### Running the simulation

In [ ]:
import math
STEP_SIZE_TRAIN=math.ceil(train_generator.n / train_generator.batch_size)
STEP_SIZE_VALID=math.ceil(validation_generator.n / validation_generator.batch_size)

history = model.fit(
    train_generator,
    steps_per_epoch=(STEP_SIZE_TRAIN),  # 2000 images = batch_size * steps
    epochs=38,
    validation_data=validation_generator,
    validation_steps=(STEP_SIZE_VALID),  # 1000 images = batch_size * steps
    verbose=2,workers=1,use_multiprocessing=False,class_weight=class_weight)


We will  plot 10 images at random from test set, but with titles as classified by the model, with every correct classification titled in 'green' color, and every incorrect classification titles in 'red' color.

In [ ]:
np.random.seed(87)

X_test, labelsfromdata = next(validation_generator)
Y_pred = np.round(model.predict(X_test))

y_pred = np.argmax(Y_pred, axis=1)
labels=(validation_generator.class_indices)
labels2=dict((v,k) for k,v in labels.items())
print(labels2)

for rand_num in np.random.randint(0, len(labels), 6):
    plt.figure()
    plt.imshow(X_test[rand_num]), plt.axis('off')
    if np.where(Y_pred[rand_num] == 1)[0].sum() == np.where(labelsfromdata[rand_num] == 1)[0].sum():
        plt.title(labels2[np.where(Y_pred[rand_num] == 1)[0].sum()], color='g')
    else :
        plt.title(labels2[np.where(Y_pred[rand_num] == 1)[0].sum()], color='r')
        print('Correct Label',labels2[np.where(labelsfromdata[rand_num] == 1)[0].sum()])


### Check Results

In [ ]:
labels = ['XR_ELBOW','XR_FINGER','XR_FOREARM','XR_HAND','XR_HUMERUS','XR_SHOULDER','XR_WRIST']
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
print("Confusion Matrix for valid Set:\n========================================")
validation_generator.reset()
Y_pred = model.predict_generator(validation_generator, validation_generator.samples // batch_size+1)

y_pred = np.argmax(Y_pred, axis=1)
labels=(validation_generator.class_indices)
labels2=dict((v,k) for k,v in labels.items())

predictions=[labels2[k] for k in y_pred]
cf_matrix = confusion_matrix(validation_generator.classes, y_pred)
print(cf_matrix)


import seaborn as sns
plt.figure()
print('Classification Report for Valid Set')
print(classification_report(validation_generator.classes, y_pred, target_names=labels))
print("Confusion Matrix for Train Set:\n========================================")
train_generator.reset()
Y_pred_T = model.predict_generator(train_generator, (train_generator.samples // batch_size+1))
y_pred_T = np.argmax(Y_pred_T, axis=1)
cf_matrix_T = confusion_matrix(train_generator.classes, y_pred_T)
print(cf_matrix_T)


print('Classification Report for Train Set')
print(classification_report(train_generator.classes, y_pred_T, target_names=labels))

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
print(history.history.keys())
acc=history.history['categorical_accuracy']

val_acc=history.history['val_categorical_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

# ------------------------------------------------
# Plot training and validation accuracy per epoch
# ------------------------------------------------
plt.figure()
plt.semilogy(epochs, acc, 'r', "Training Accuracy")
plt.semilogy(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation Loss')
plt.title('Model Accuracy', weight='bold', fontsize=16)
plt.ylabel('accuracy', weight='bold', fontsize=14)
plt.xlabel('epoch', weight='bold', fontsize=14)
plt.grid(color = 'y', linewidth='0.5')
plt.legend()
# ------------------------------------------------
# Plot training and validation accuracy per epoch
# ------------------------------------------------
plt.figure()
plt.semilogy(epochs, loss, 'r', "Loss of Training")
plt.semilogy(epochs, val_loss, 'b', "Loss of Validation")
plt.title('Training and validation Loss')
plt.title('Loss of Training and Validation Set', weight='bold', fontsize=16)
plt.ylabel('Loss ', weight='bold', fontsize=14)
plt.grid(color = 'y', linewidth='0.5')
plt.xlabel('epoch', weight='bold', fontsize=14)
plt.legend()